# Importing the libraries

In [24]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import time

# API key and ticker symbol

In [25]:
api_key = 'HDHS7RJSLYNN07J7'
ticker = 'TSLA'

# Importing the dataset

In [26]:
ts = TimeSeries(key = api_key, output_format = 'pandas')
data, meta_data = ts.get_intraday(symbol=ticker, interval='1min', outputsize='full')
print(data.head())

                       1. open    2. high     3. low   4. close  5. volume
date                                                                      
2020-07-15 20:00:00  1511.0000  1511.0000  1511.0000  1511.0000     2920.0
2020-07-15 19:59:00  1511.2200  1513.0000  1510.9500  1512.5000     2390.0
2020-07-15 19:58:00  1510.9500  1512.0000  1510.9500  1512.0000     2279.0
2020-07-15 19:57:00  1510.5000  1510.7700  1510.1000  1510.7700     2918.0
2020-07-15 19:56:00  1510.1601  1510.1601  1510.1601  1510.1601     1400.0


# Data preprocessing

## Renaming columns

In [27]:
data.columns = ['o', 'h', 'l', 'c', 'vol']
print(data.head())

                             o          h          l          c     vol
date                                                                   
2020-07-15 20:00:00  1511.0000  1511.0000  1511.0000  1511.0000  2920.0
2020-07-15 19:59:00  1511.2200  1513.0000  1510.9500  1512.5000  2390.0
2020-07-15 19:58:00  1510.9500  1512.0000  1510.9500  1512.0000  2279.0
2020-07-15 19:57:00  1510.5000  1510.7700  1510.1000  1510.7700  2918.0
2020-07-15 19:56:00  1510.1601  1510.1601  1510.1601  1510.1601  1400.0


## Sentiment (Bearish or Bullish)

In [28]:
color = []
for ind in data.index:
    if data['o'][ind] > data['c'][ind]:
        color.append('red') #open higher than close --> bearishness
    else:
        color.append('blue') #open is lower than or equal to close --> bulllishness
data['color'] = color
print(data.head())

                             o          h          l          c     vol color
date                                                                         
2020-07-15 20:00:00  1511.0000  1511.0000  1511.0000  1511.0000  2920.0  blue
2020-07-15 19:59:00  1511.2200  1513.0000  1510.9500  1512.5000  2390.0  blue
2020-07-15 19:58:00  1510.9500  1512.0000  1510.9500  1512.0000  2279.0  blue
2020-07-15 19:57:00  1510.5000  1510.7700  1510.1000  1510.7700  2918.0  blue
2020-07-15 19:56:00  1510.1601  1510.1601  1510.1601  1510.1601  1400.0  blue


## Range

In [29]:
range = []
for ind in data.index:
    range.append(data['h'][ind] - data['l'][ind])
data['range'] = range
print(data.head())

                             o          h          l          c     vol color  \
date                                                                            
2020-07-15 20:00:00  1511.0000  1511.0000  1511.0000  1511.0000  2920.0  blue   
2020-07-15 19:59:00  1511.2200  1513.0000  1510.9500  1512.5000  2390.0  blue   
2020-07-15 19:58:00  1510.9500  1512.0000  1510.9500  1512.0000  2279.0  blue   
2020-07-15 19:57:00  1510.5000  1510.7700  1510.1000  1510.7700  2918.0  blue   
2020-07-15 19:56:00  1510.1601  1510.1601  1510.1601  1510.1601  1400.0  blue   

                     range  
date                        
2020-07-15 20:00:00   0.00  
2020-07-15 19:59:00   2.05  
2020-07-15 19:58:00   1.05  
2020-07-15 19:57:00   0.67  
2020-07-15 19:56:00   0.00  


## Shadow length

In [30]:
u_shadow = []
l_shadow = []
for ind in data.index:
    if data['color'][ind] == 'red':
        u_shadow.append(data['h'][ind] - data['o'][ind])
        l_shadow.append(data['c'][ind] - data['l'][ind])
    else:
        u_shadow.append(data['h'][ind] - data['c'][ind])
        l_shadow.append(data['o'][ind] - data['l'][ind])
data['u_shadow'] = u_shadow
data['l_shadow'] = l_shadow
print(data.head())

                             o          h          l          c     vol color  \
date                                                                            
2020-07-15 20:00:00  1511.0000  1511.0000  1511.0000  1511.0000  2920.0  blue   
2020-07-15 19:59:00  1511.2200  1513.0000  1510.9500  1512.5000  2390.0  blue   
2020-07-15 19:58:00  1510.9500  1512.0000  1510.9500  1512.0000  2279.0  blue   
2020-07-15 19:57:00  1510.5000  1510.7700  1510.1000  1510.7700  2918.0  blue   
2020-07-15 19:56:00  1510.1601  1510.1601  1510.1601  1510.1601  1400.0  blue   

                     range  u_shadow  l_shadow  
date                                            
2020-07-15 20:00:00   0.00       0.0      0.00  
2020-07-15 19:59:00   2.05       0.5      0.27  
2020-07-15 19:58:00   1.05       0.0      0.00  
2020-07-15 19:57:00   0.67       0.0      0.40  
2020-07-15 19:56:00   0.00       0.0      0.00  


## Upper bound and lower bound

In [31]:
ub = []
lb = []
for ind in data.index:
    if data['color'][ind] == 'blue':
        ub.append(data['c'][ind])
        lb.append(data['o'][ind])
    else:
        ub.append(data['o'][ind])
        lb.append(data['c'][ind])
data['u_bond'] = ub
data['l_bound'] = lb
print(data.head())

                             o          h          l          c     vol color  \
date                                                                            
2020-07-15 20:00:00  1511.0000  1511.0000  1511.0000  1511.0000  2920.0  blue   
2020-07-15 19:59:00  1511.2200  1513.0000  1510.9500  1512.5000  2390.0  blue   
2020-07-15 19:58:00  1510.9500  1512.0000  1510.9500  1512.0000  2279.0  blue   
2020-07-15 19:57:00  1510.5000  1510.7700  1510.1000  1510.7700  2918.0  blue   
2020-07-15 19:56:00  1510.1601  1510.1601  1510.1601  1510.1601  1400.0  blue   

                     range  u_shadow  l_shadow     u_bond    l_bound  
date                                                                  
2020-07-15 20:00:00   0.00       0.0      0.00  1511.0000  1511.0000  
2020-07-15 19:59:00   2.05       0.5      0.27  1512.5000  1511.2200  
2020-07-15 19:58:00   1.05       0.0      0.00  1512.0000  1510.9500  
2020-07-15 19:57:00   0.67       0.0      0.40  1510.7700  1510.5000  
2020-0

# Single candlestick patterns

## Marubozu

In [32]:
type = []
data_dict = data.to_dict('records')
for r in data_dict:
    if (r('u_shadow')/r('u_bound') <= 0.003 and 
        r('l_shadow')/r('l_bound') <= 0.003):
        type.append('marubozu')

TypeError: 'dict' object is not callable